In [3]:
import requests
from bs4 import BeautifulSoup
import csv

In [4]:
# Scraper to get list of recent sales from Oklahoma County Assessor

def get_sales_list(year='2019'):
    # We need a session to go between pages properly. Otherwise it goes like: 1, 2, 2, 3, 2, 3, 2, ... forever
    sesh = requests.session()
    # get the first page
    req = sesh.get("https://ariisp1.oklahomacounty.org/AssessorWP5/SalesSearch.asp",
                  data={'fpdbr_3_PagingMove': '  |<  ',
                       'SaleDate': year,
                       'Deed': "DE"})
    data = req.text

    done = False
    sales_list = []
    while not done:
        the_soup = BeautifulSoup(data, features="lxml")
        rows = the_soup.find_all('table')[3].tbody.find_all('tr')

        for r in rows:
            cur_tds = r.find_all('td')
            sale_dict = {}

            #print(r)
            if not r.find_all('form'): # Verify we're not on the last row (which has the form tag)
                sale_dict['date'] = cur_tds[0].font.text.strip()
                sale_dict['account_number'] = cur_tds[2].font.text.strip()
                price_str = cur_tds[8].font.text.strip()
                sale_dict['price'] = int(price_str.replace(',', '')) if price_str != "" else 0

                sales_list.append(sale_dict)
            else:
                print("finished page - " + str(len(sales_list)) + " rows")
                #fpdbr_4_PagingMove
                if r.find_all('input', attrs={'value' : "  >   "}):
                    req = sesh.get("https://ariisp1.oklahomacounty.org/AssessorWP5/SalesSearch.asp",
                                  data={'fpdbr_4_PagingMove': "  >   ",
                                       'SaleDate': year,
                                       'Deed': "DE"})
                    data = req.text
                else:
                    done = True
    return sales_list

In [6]:
year = '2017'
output_filename = 'sales_list_' + year + '.csv'

sales_list = get_sales_list(year)
with open(output_filename, 'w') as csvfile:
    fieldnames = ['date', 'account_number', 'price']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for r in sales_list:
        writer.writerow(r)

finished page - 100 rows
finished page - 200 rows
finished page - 300 rows
finished page - 400 rows
finished page - 500 rows
finished page - 600 rows
finished page - 700 rows
finished page - 800 rows
finished page - 900 rows
finished page - 1000 rows
finished page - 1100 rows
finished page - 1200 rows
finished page - 1300 rows
finished page - 1400 rows
finished page - 1500 rows
finished page - 1600 rows
finished page - 1700 rows
finished page - 1800 rows
finished page - 1900 rows
finished page - 2000 rows
finished page - 2100 rows
finished page - 2200 rows
finished page - 2300 rows
finished page - 2400 rows
finished page - 2500 rows
finished page - 2600 rows
finished page - 2700 rows
finished page - 2800 rows
finished page - 2900 rows
finished page - 3000 rows
finished page - 3100 rows
finished page - 3200 rows
finished page - 3300 rows
finished page - 3400 rows
finished page - 3500 rows
finished page - 3600 rows
finished page - 3700 rows
finished page - 3800 rows
finished page - 3900 